## Modelo datos estructurados
Este notebook desarrolla un primer modelo para resolver el problema de Petfinder. Empezamos haciendo un modelo inicial muy simple para ver la viabilidad de resolver el problema. Luego analizamos como se comporta la métrica kappa propuesta y vemos la matriz de confusión. Finalmente hacemos una optimizacin de hiperparametros evaluando con train/test y otra validando con 5 fold CV y testeando en el 20% de los datos

In [1]:
#Import de librerias basicas tablas y matrices
import numpy as np 
import pandas as pd 


#Gradient Boosting
import lightgbm as lgb

#Funciones auxiliares sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, StratifiedKFold #Split y cross Validation
from sklearn.metrics import cohen_kappa_score, accuracy_score, balanced_accuracy_score #Metricas
from sklearn.preprocessing import OneHotEncoder

# from sklearn.utils import shuffle 

#Visualizacióon
from plotly import express as px

#Plot de matriz de confusion normalizada en actuals
from utils import plot_confusion_matrix

import os

#Optimizacion de hiperparametros
import optuna
from optuna.artifacts import FileSystemArtifactStore, upload_artifact

#Guardado de objetos en archivos joblib
from joblib import load, dump



In [2]:
#PONER LA ESTRUCUTRA DE CARPETAS COMO SE VE AQUI ABAJO !!!

In [3]:
# Paths para acceso archivos
#Este notebook asume la siguiente estructura de carpetas a partir de la ubicacion de base_dir 
#(dos niveles arriba de la carpeta donde se ejecuta el notebook). 
# /UA_MDM_LDI_II/
# /UA_MDM_LDI_II/input
# /UA_MDM_LDI_II/input/petfinder-adoption-prediction/            <- Aca deben ir todos los archivos de datos de la competencia 
# /UA_MDM_LDI_II/tutoriales/                       <- Aca deben poner los notebooks y scripts que les compartimos
# /UA_MDM_LDI_II/work/                             <- Resultados de notebooks iran dentro de esta carpeta en subcarpetas
# /UA_MDM_LDI_II/work/models/                     <- Modelos entrenados en archivos joblibs
# /UA_MDM_LDI_II/work/optuna_temp_artifacts/      <- Archivos que queremos dejar como artefacto de un trial de optuna (optuna los copiara a la carpeta de abajo)
# /UA_MDM_LDI_II/work/optuna_artifacts/           <- Archivos con artefactos que sibimos a optuna

#Subimos dos niveles para quedar en la carpeta que contiene input y UA_MDM_LDI_II
BASE_DIR = ''

#Datos de entrenamiento 
PATH_TO_TRAIN = os.path.join(BASE_DIR, "input/petfinder-adoption-prediction/train/train.csv")

#Datos de test
PATH_TO_TEST = os.path.join(BASE_DIR, "input/petfinder-adoption-prediction/test/test.csv")

#Salida de modelos entrenados
PATH_TO_MODELS = os.path.join(BASE_DIR, "work/models")

#Artefactos a subir a optuna
PATH_TO_TEMP_FILES = os.path.join(BASE_DIR, "work/optuna_temp_artifacts")

#Artefactos que optuna gestiona
PATH_TO_OPTUNA_ARTIFACTS = os.path.join(BASE_DIR, "work/optuna_artifacts")


SEED = 42 #Semilla de procesos aleatorios (para poder replicar exactamente al volver a correr un modelo)
TEST_SIZE = 0.2 #Facción para train/test= split 

#PONER CUALQUIER SEMILLLA Y 20% DEL DATASTE USAMOS

In [4]:
#NUESTRO TODO ES TRAIN ... ENTONCES EN BASE A ESE DATASET HACEMOS TRAIN Y TEST

In [5]:
# Datos Tabulares
dataset = pd.read_csv(PATH_TO_TRAIN)

In [6]:
#Columnas del dataset
dataset.columns

Index(['Type', 'Name', 'Age', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2',
       'Color3', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed',
       'Sterilized', 'Health', 'Quantity', 'Fee', 'State', 'RescuerID',
       'VideoAmt', 'Description', 'PetID', 'PhotoAmt', 'AdoptionSpeed'],
      dtype='object')

In [7]:
dataset.head()

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,...,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,AdoptionSpeed
0,2,Nibble,3,299,0,1,1,7,0,1,...,1,1,100,41326,8480853f516546f6cf33aa88cd76c379,0,Nibble is a 3+ month old ball of cuteness. He ...,86e1089a3,1.0,2
1,2,No Name Yet,1,265,0,1,1,2,0,2,...,1,1,0,41401,3082c7125d8fb66f7dd4bff4192c8b14,0,I just found it alone yesterday near my apartm...,6296e909a,2.0,0
2,1,Brisco,1,307,0,1,2,7,0,2,...,1,1,0,41326,fa90fa5b1ee11c86938398b60abc32cb,0,Their pregnant mother was dumped by her irresp...,3422e4906,7.0,3
3,1,Miko,4,307,0,2,1,2,0,2,...,1,1,150,41401,9238e4f44c71a75282e62f7136c6b240,0,"Good guard dog, very alert, active, obedience ...",5842f1ff5,8.0,2
4,1,Hunter,1,307,0,1,1,0,0,2,...,1,1,0,41326,95481e953f8aed9ec3d16fc4509537e8,0,This handsome yet cute boy is up for adoption....,850a43f90,3.0,2


## FEATURE ENGENEERING
Haremos modificaciones en el dataset para ver si mejora o no la fuerza predictiva del modelo base

In [8]:
#Imputar datos faltantes en Nombre
dataset.isnull().sum()

Type                0
Name             1265
Age                 0
Breed1              0
Breed2              0
Gender              0
Color1              0
Color2              0
Color3              0
MaturitySize        0
FurLength           0
Vaccinated          0
Dewormed            0
Sterilized          0
Health              0
Quantity            0
Fee                 0
State               0
RescuerID           0
VideoAmt            0
Description        13
PetID               0
PhotoAmt            0
AdoptionSpeed       0
dtype: int64

In [9]:
#FE VIEJO 

def feature_engineering(dataset):
    # Categorizacion de edad
    bins = [0, 12, 60, float('inf')]  # Define la cantidad de categorias y los limites de las mismas
    labels = [1, 2, 3]  # 1='puppy' 2='adult' 3='senior'
    dataset['AgeCategory'] = pd.cut(dataset['Age'], bins, labels=labels, right=False)

    # Combinacion para Breed
    dataset['Breed'] = dataset.apply(lambda row: 2 if row['Breed2'] != 0 else 1, axis=1)

    # Color
    dataset['IsMultiColored'] = dataset.apply(lambda row: 1 if row['Color2'] != 0 or row['Color3'] != 0 else 0, axis=1)

    # Name
    dataset['Name'].fillna('No Name Yet', inplace=True)
    dataset['Name'] = dataset['Name'].replace('', 'No Name Yet')
    dataset['name_code'] = dataset['Name'].apply(lambda x: 1 if x != 'No Name Yet' else 0)

    # Creación de nuevas características
    dataset['Age_log'] = np.log1p(dataset['Age'])  # logaritmo de la edad
    dataset['PhotoAmt_log'] = np.log1p(dataset['PhotoAmt'])  # logaritmo de la cantidad de fotos
    dataset['VideoAmt_log'] = np.log1p(dataset['VideoAmt'])  # logaritmo de la cantidad de videos

    # Interacciones entre características
    dataset['Age_PhotoAmt'] = dataset['Age'] * dataset['PhotoAmt']  # relación entre la edad de la mascota y la cantidad de fotos
    dataset['Age_VideoAmt'] = dataset['Age'] * dataset['VideoAmt']  # relación entre la edad de la mascota y la cantidad de videos

    # Agregaciones y agrupaciones
    dataset['Breed1_mean_Age'] = dataset.groupby('Breed1')['Age'].transform('mean')  # promedio de edad por raza
    dataset['Breed1_mean_PhotoAmt'] = dataset.groupby('Breed1')['PhotoAmt'].transform('mean')  # promedio de fotos por raza

    # Agregar cantidad de palabras del describe y longitud del texto
    dataset["description_word_count"] = dataset["Description"].apply(lambda x: len(str(x).split()) if pd.notnull(x) else 0)
    dataset['Description_Length'] = dataset['Description'].apply(lambda x: len(x) if isinstance(x, str) else 0)

    # Health
    dataset['Health'] = dataset['Vaccinated'].astype(str) + '_' + dataset['Dewormed'].astype(str) + '_' + dataset['Sterilized'].astype(str)

    # Nuevas características basadas en interacciones y transformaciones
    dataset['Photo_Video_Ratio'] = dataset['PhotoAmt'] / (dataset['VideoAmt'] + 1)  # Evitar división por cero
    dataset['Fee_log'] = np.log1p(dataset['Fee'])  # Logaritmo de la tarifa de adopción
    dataset['Fee_Age_Ratio'] = dataset['Fee'] / (dataset['Age'] + 1)  # Evitar división por cero
    dataset['RescuerID_count'] = dataset.groupby('RescuerID')['RescuerID'].transform('count')  # Número de mascotas rescatadas por el mismo rescatador
    dataset['Name_Length'] = dataset['Name'].apply(len)  # Longitud del nombre de la mascota
    dataset['Name_Word_Count'] = dataset['Name'].apply(lambda x: len(x.split()))  # Número de palabras en el nombre de la mascota
    dataset['RescuerID_Length'] = dataset['RescuerID'].apply(len)  # Longitud del ID del rescatador
    dataset['RescuerID_Word_Count'] = dataset['RescuerID'].apply(lambda x: len(x.split()))  # Número de palabras en el ID del rescatador
    dataset['MaturitySize_FurLength'] = dataset['MaturitySize'] * dataset['FurLength']
    dataset['PhotoAmt_FurLength'] = dataset['PhotoAmt'] * dataset['FurLength']
    dataset['VideoAmt_FurLength'] = dataset['VideoAmt'] * dataset['FurLength']

    return dataset


In [83]:
#FE NUEVO
def feature_engineering(dataset):
    # Age categorization
    bins = [0, 12, 60, float('inf')]
    labels = [1, 2, 3]  # 1='puppy' 2='adult' 3='senior'
    dataset['AgeCategory'] = pd.cut(dataset['Age'], bins, labels=labels, right=False)

    # Breed combination
    dataset['Breed'] = dataset.apply(lambda row: 2 if row['Breed2'] != 0 else 1, axis=1)

    # Color
    dataset['IsMultiColored'] = dataset.apply(lambda row: 1 if row['Color2'] != 0 or row['Color3'] != 0 else 0, axis=1)

    # Name processing
    dataset['Name'].fillna('No Name Yet', inplace=True) # Fill missing values
    dataset['Name'] = dataset['Name'].replace('', 'No Name Yet') # Replace empty strings
    dataset['name_code'] = dataset['Name'].apply(lambda x: 1 if x != 'No Name Yet' else 0) # Create a binary variable for name presence
    dataset['Is_Genuine_Name'] = ((dataset['Name'] != 'No Name Yet') & (dataset['Quantity'] > 1)).astype(int)    #geniuine name

    # Logarithmic transformations and means
    dataset['Age_log'] = np.log1p(dataset['Age'])
    dataset['PhotoAmt_log'] = np.log1p(dataset['PhotoAmt'])
    dataset['VideoAmt_log'] = np.log1p(dataset['VideoAmt'])
    dataset['Fee_log'] = np.log1p(dataset['Fee'])
    # Interactions and aggregations by gender
    dataset['Gender_Age_mean'] = dataset.groupby('Gender')['Age'].transform('mean')
    dataset['Gender_Fee_mean'] = dataset.groupby('Gender')['Fee'].transform('mean')
    dataset['Gender_PhotoAmt_mean'] = dataset.groupby('Gender')['PhotoAmt'].transform('mean')
    dataset['Gender_VideoAmt_mean'] = dataset.groupby('Gender')['VideoAmt'].transform('mean')
    # Interactions and aggregations by breed (assuming Breed1 is the primary breed)
    dataset['Breed_Age_mean'] = dataset.groupby('Breed1')['Age'].transform('mean')
    dataset['Breed_Fee_mean'] = dataset.groupby('Breed1')['Fee'].transform('mean')
    dataset['Breed_PhotoAmt_mean'] = dataset.groupby('Breed1')['PhotoAmt'].transform('mean')
    dataset['Breed_VideoAmt_mean'] = dataset.groupby('Breed1')['VideoAmt'].transform('mean')
    # Interactions between gender and breed
    dataset['Gender_Breed_Age_mean'] = dataset.groupby(['Gender', 'Breed1'])['Age'].transform('mean')
    dataset['Gender_Breed_Fee_mean'] = dataset.groupby(['Gender', 'Breed1'])['Fee'].transform('mean')
    dataset['Gender_Breed_PhotoAmt_mean'] = dataset.groupby(['Gender', 'Breed1'])['PhotoAmt'].transform('mean')
    dataset['Gender_Breed_VideoAmt_mean'] = dataset.groupby(['Gender', 'Breed1'])['VideoAmt'].transform('mean')
    # Ratios and differences
    dataset['Age_to_Gender_mean_ratio'] = dataset['Age'] / dataset['Gender_Age_mean']
    dataset['Fee_to_Gender_mean_ratio'] = dataset['Fee'] / dataset['Gender_Fee_mean']
    dataset['Age_to_Breed_mean_ratio'] = dataset['Age'] / dataset['Breed_Age_mean']
    dataset['Fee_to_Breed_mean_ratio'] = dataset['Fee'] / dataset['Breed_Fee_mean']
    # Interaction between continuous variables
    dataset['Age_Fee_interaction'] = dataset['Age'] * dataset['Fee']
    dataset['PhotoAmt_VideoAmt_interaction'] = dataset['PhotoAmt'] * dataset['VideoAmt']
    # Logarithmic transformations of the new features
    dataset['Gender_Fee_mean_log'] = np.log1p(dataset['Gender_Fee_mean'])
    dataset['Breed_Fee_mean_log'] = np.log1p(dataset['Breed_Fee_mean'])
    dataset['Gender_Breed_Fee_mean_log'] = np.log1p(dataset['Gender_Breed_Fee_mean'])
    # Additional complex interactions
    dataset['Age_PhotoAmt_Fee_interaction'] = dataset['Age'] * dataset['PhotoAmt'] * dataset['Fee']
    dataset['Age_VideoAmt_Fee_interaction'] = dataset['Age'] * dataset['VideoAmt'] * dataset['Fee']

    # Feature interactions
    dataset['Age_PhotoAmt'] = dataset['Age'] * dataset['PhotoAmt'] # Interaction between Age and PhotoAmt
    dataset['Age_VideoAmt'] = dataset['Age'] * dataset['VideoAmt'] # Interaction between Age and VideoAmt
    dataset['Photo_Video_Ratio'] = dataset['PhotoAmt'] / (dataset['VideoAmt'] + 1) # Ratio between PhotoAmt and VideoAmt
    dataset['Fee_Age_Ratio'] = dataset['Fee'] / (dataset['Age'] + 1) # Ratio between Fee and Age
    dataset['MaturitySize_FurLength'] = dataset['MaturitySize'] * dataset['FurLength'] # Interaction between MaturitySize and FurLength
    dataset['PhotoAmt_FurLength'] = dataset['PhotoAmt'] * dataset['FurLength'] # Interaction between PhotoAmt and FurLength
    dataset['VideoAmt_FurLength'] = dataset['VideoAmt'] * dataset['FurLength'] # Interaction between VideoAmt and FurLength

    # Aggregations and groupings
    dataset['Breed1_mean_Age'] = dataset.groupby('Breed1')['Age'].transform('mean') # Mean Age by Breed1
    dataset['Breed1_mean_PhotoAmt'] = dataset.groupby('Breed1')['PhotoAmt'].transform('mean') # Mean PhotoAmt by Breed1
    dataset['RescuerID_count'] = dataset.groupby('RescuerID')['RescuerID'].transform('count') # Count of pets by RescuerID

    # Text-based features
    dataset["description_word_count"] = dataset["Description"].apply(lambda x: len(str(x).split()) if pd.notnull(x) else 0) # Word count in Description
    dataset['Description_Length'] = dataset['Description'].apply(lambda x: len(x) if isinstance(x, str) else 0) # Length of Description
    dataset['Name_Length'] = dataset['Name'].apply(len) # Length of Name
    dataset['Name_Word_Count'] = dataset['Name'].apply(lambda x: len(x.split())) # Word count in Name
    dataset['Name_Length_Category'] = dataset['Name'].apply(lambda x: 0 if len(x) < 5 else 1) # Categorization of Name length

    # Health


    # Single condition features
    dataset['Health_vac'] = np.where(dataset['Vaccinated'] == 1, 1, 0)
    dataset['Health_dew'] = np.where(dataset['Dewormed'] == 1, 1, 0)
    dataset['Health_ster'] = np.where(dataset['Sterilized'] == 1, 1, 0)
    # Two condition combinations
    dataset['Health_vac_dew'] = np.where((dataset['Vaccinated'] == 1) & (dataset['Dewormed'] == 1), 1, 0)
    dataset['Health_vac_ster'] = np.where((dataset['Vaccinated'] == 1) & (dataset['Sterilized'] == 1), 1, 0)
    dataset['Health_dew_ster'] = np.where((dataset['Dewormed'] == 1) & (dataset['Sterilized'] == 1), 1, 0)
    # All three conditions (full health)
    dataset['Health_full'] = np.where((dataset['Vaccinated'] == 1) & (dataset['Dewormed'] == 1) & (dataset['Sterilized'] == 1), 1, 0)
    # Inverse conditions (not done)
    dataset['Health_not_vac'] = np.where(dataset['Vaccinated'] == 0, 1, 0)
    dataset['Health_not_dew'] = np.where(dataset['Dewormed'] == 0, 1, 0)
    dataset['Health_not_ster'] = np.where(dataset['Sterilized'] == 0, 1, 0)
    # Two inverse condition combinations
    dataset['Health_not_vac_dew'] = np.where((dataset['Vaccinated'] == 0) & (dataset['Dewormed'] == 0), 1, 0)
    dataset['Health_not_vac_ster'] = np.where((dataset['Vaccinated'] == 0) & (dataset['Sterilized'] == 0), 1, 0)
    dataset['Health_not_dew_ster'] = np.where((dataset['Dewormed'] == 0) & (dataset['Sterilized'] == 0), 1, 0)
    # No health procedures done
    dataset['Health_none'] = np.where((dataset['Vaccinated'] == 0) & (dataset['Dewormed'] == 0) & (dataset['Sterilized'] == 0), 1, 0)
    
    # Additional RescuerID-based features
    
    # Average age of pets for each rescuer
    dataset['RescuerID_mean_Age'] = dataset.groupby('RescuerID')['Age'].transform('mean')
    
    # Standard deviation of pet ages for each rescuer
    dataset['RescuerID_std_Age'] = dataset.groupby('RescuerID')['Age'].transform('std')
    
    # Average fee for each rescuer
    dataset['RescuerID_mean_Fee'] = dataset.groupby('RescuerID')['Fee'].transform('mean')
    
    # Standard deviation of fees for each rescuer
    dataset['RescuerID_std_Fee'] = dataset.groupby('RescuerID')['Fee'].transform('std')
    
    # Average number of photos per pet for each rescuer
    dataset['RescuerID_mean_PhotoAmt'] = dataset.groupby('RescuerID')['PhotoAmt'].transform('mean')
    
    # Average number of videos per pet for each rescuer
    dataset['RescuerID_mean_VideoAmt'] = dataset.groupby('RescuerID')['VideoAmt'].transform('mean')
    
    # Percentage of pets with names for each rescuer
    dataset['RescuerID_name_ratio'] = dataset.groupby('RescuerID')['name_code'].transform('mean')
    
    # Most common breed for each rescuer
    dataset['RescuerID_most_common_breed'] = dataset.groupby('RescuerID')['Breed1'].transform(lambda x: x.value_counts().index[0])
    
    # Percentage of pets that are puppies/kittens for each rescuer
    dataset['RescuerID_young_ratio'] = dataset.groupby('RescuerID')['AgeCategory'].transform(lambda x: (x == 1).mean())
    
    # Percentage of pets with full health (vaccinated, dewormed, sterilized) for each rescuer
    dataset['RescuerID_full_health_ratio'] = dataset.groupby('RescuerID')['Health_full'].transform('mean')
    
    # Average description length for each rescuer
    dataset['RescuerID_mean_description_length'] = dataset.groupby('RescuerID')['Description_Length'].transform('mean')
    
    # Time since first listing for each rescuer (assuming 'DateTime' column exists)
    if 'DateTime' in dataset.columns:
        dataset['RescuerID_days_active'] = dataset.groupby('RescuerID')['DateTime'].transform(lambda x: (x.max() - x.min()).days)
    
    # Rescuer's pet type preference (dog vs cat ratio)
    dataset['RescuerID_dog_ratio'] = dataset.groupby('RescuerID')['Type'].transform(lambda x: (x == 1).mean())
    
    # Rescuer's average sentiment score (if you have sentiment analysis on descriptions)
    # This is a placeholder - you'd need to implement sentiment analysis first
    # dataset['RescuerID_mean_sentiment'] = dataset.groupby('RescuerID')['sentiment_score'].transform('mean')
    
    # Rescuer's specialization score (how much they focus on specific breeds)
    dataset['RescuerID_breed_specialization'] = dataset.groupby('RescuerID')['Breed1'].transform(lambda x: x.value_counts().iloc[0] / len(x))
    
    return dataset

In [84]:
feature_engineering(dataset)
dataset.head()

C:\Users\s1093678\AppData\Local\Temp\ipykernel_18756\3350703556.py:14: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,...,RescuerID_std_Fee,RescuerID_mean_PhotoAmt,RescuerID_mean_VideoAmt,RescuerID_name_ratio,RescuerID_most_common_breed,RescuerID_young_ratio,RescuerID_full_health_ratio,RescuerID_mean_description_length,RescuerID_dog_ratio,RescuerID_breed_specialization
0,2,Nibble,3,299,0,1,1,7,0,1,...,51.754917,1.500000,0.000000,1.000000,276,0.750000,0.000000,434.250000,0.000000,0.375000
1,2,No Name Yet,1,265,0,1,1,2,0,2,...,NaN,2.000000,0.000000,0.000000,265,1.000000,0.000000,118.000000,0.000000,1.000000
2,1,Brisco,1,307,0,1,2,7,0,2,...,14.927688,3.572985,0.010893,0.986928,307,0.941176,0.106754,174.178649,0.991285,0.684096
3,1,Miko,4,307,0,2,1,2,0,2,...,70.101967,9.500000,0.020000,1.000000,307,0.820000,0.280000,426.180000,0.820000,0.760000
4,1,Hunter,1,307,0,1,1,0,0,2,...,0.000000,2.305970,0.007463,1.000000,307,0.895522,0.044776,242.029851,0.970149,0.895522


In [85]:
# Supongamos que 'dataset' es tu DataFrame

# Listar variables numéricas
numeric_features = dataset.select_dtypes(include=['number']).columns.tolist()
print("Variables numéricas:", numeric_features)

# Listar variables categóricas
categorical_features = dataset.select_dtypes(include=['object', 'category']).columns.tolist()
print("Variables categóricas:", categorical_features)

Variables numéricas: ['Type', 'Age', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2', 'Color3', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed', 'Sterilized', 'Health', 'Quantity', 'Fee', 'State', 'VideoAmt', 'PhotoAmt', 'AdoptionSpeed', 'Breed', 'IsMultiColored', 'name_code', 'Is_Genuine_Name', 'Age_log', 'PhotoAmt_log', 'VideoAmt_log', 'Fee_log', 'Gender_Age_mean', 'Gender_Fee_mean', 'Gender_PhotoAmt_mean', 'Gender_VideoAmt_mean', 'Breed_Age_mean', 'Breed_Fee_mean', 'Breed_PhotoAmt_mean', 'Breed_VideoAmt_mean', 'Gender_Breed_Age_mean', 'Gender_Breed_Fee_mean', 'Gender_Breed_PhotoAmt_mean', 'Gender_Breed_VideoAmt_mean', 'Age_to_Gender_mean_ratio', 'Fee_to_Gender_mean_ratio', 'Age_to_Breed_mean_ratio', 'Fee_to_Breed_mean_ratio', 'Age_Fee_interaction', 'PhotoAmt_VideoAmt_interaction', 'Gender_Fee_mean_log', 'Breed_Fee_mean_log', 'Gender_Breed_Fee_mean_log', 'Age_PhotoAmt_Fee_interaction', 'Age_VideoAmt_Fee_interaction', 'Age_PhotoAmt', 'Age_VideoAmt', 'Photo_Video_Ratio', 'Fee_Ag

In [86]:
#Este fragmento de código define una función add_new_variables_with_lgbm que agrega nuevas variables a un conjunto de datos utilizando un modelo de LightGBM. Aquí está el desglose paso a paso:
def add_new_variables_with_lgbm(dataset):
    # Selección de características numéricas y categóricas:
    numeric_features = ['Age', 'VideoAmt', 'PhotoAmt', 'AgeCategory', 'IsMultiColored', 'name_code','Age_log', 'PhotoAmt_log', 'VideoAmt_log', 'Age_PhotoAmt', 'Age_VideoAmt', 'Breed1_mean_Age', 'Breed1_mean_PhotoAmt']
    categorical_features = ['Type', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2', 'Color3']

    # Codificación de características categóricas:
    encoded_features = pd.get_dummies(dataset[categorical_features], columns=categorical_features, prefix=categorical_features)

    # Concatenación de características numéricas y codificadas:
    processed_dataset = pd.concat([dataset[numeric_features], encoded_features], axis=1)

    # Creación del conjunto de datos de entrenamiento:
    target = dataset['AdoptionSpeed']
    train_data = lgb.Dataset(processed_dataset, label=target, params={'verbose': -1})

    # Definición de parámetros de LightGBM
    lgb_params = {
        'objective': 'multiclass',
        'num_class': 5,  
        'metric': 'multi_logloss',
        'boosting_type': 'gbdt',
    }

    # Entrenamiento del modelo LightGBM:
    model = lgb.train(lgb_params, train_data)

    # Obtención de predicciones para cada muestra:
    predictions = model.predict(processed_dataset)

    # Adición de nuevas variables basadas en las predicciones:
    for class_num in range(5):  
        dataset[f'prediction_class_{class_num}'] = predictions[:, class_num]

    return dataset

# Llamada a la función con el DataFrame:
df_with_new_variables = add_new_variables_with_lgbm(dataset.copy())  # Usar dataset.copy() para evitar modificar el DataFrame original

In [87]:
df_with_new_variables

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,...,RescuerID_std_Fee,RescuerID_mean_PhotoAmt,RescuerID_mean_VideoAmt,RescuerID_name_ratio,RescuerID_most_common_breed,RescuerID_young_ratio,RescuerID_full_health_ratio,RescuerID_mean_description_length,RescuerID_dog_ratio,RescuerID_breed_specialization
0,2,Nibble,3,299,0,1,1,7,0,1,...,51.754917,1.500000,0.000000,1.000000,276,0.750000,0.000000,434.250000,0.000000,0.375000
1,2,No Name Yet,1,265,0,1,1,2,0,2,...,NaN,2.000000,0.000000,0.000000,265,1.000000,0.000000,118.000000,0.000000,1.000000
2,1,Brisco,1,307,0,1,2,7,0,2,...,14.927688,3.572985,0.010893,0.986928,307,0.941176,0.106754,174.178649,0.991285,0.684096
3,1,Miko,4,307,0,2,1,2,0,2,...,70.101967,9.500000,0.020000,1.000000,307,0.820000,0.280000,426.180000,0.820000,0.760000
4,1,Hunter,1,307,0,1,1,0,0,2,...,0.000000,2.305970,0.007463,1.000000,307,0.895522,0.044776,242.029851,0.970149,0.895522
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14988,2,No Name Yet,2,266,0,3,1,0,0,2,...,0.000000,3.000000,0.000000,0.000000,265,1.000000,0.000000,396.500000,0.000000,0.500000
14989,2,Serato & Eddie,60,265,264,3,1,4,7,2,...,0.000000,3.333333,0.000000,1.000000,266,0.000000,1.000000,142.000000,0.000000,0.666667
14990,2,Monkies,2,265,266,3,5,6,7,3,...,9.642741,3.368421,0.000000,1.000000,266,0.578947,0.526316,211.210526,0.263158,0.315789
14991,2,Ms Daym,9,266,0,2,4,7,0,1,...,0.000000,2.600000,0.000000,1.000000,254,1.000000,0.200000,266.800000,0.000000,0.600000


In [88]:
dataset=df_with_new_variables

In [89]:

#Separo un 20% para test estratificado opr target
#PERO MANTENEMOS LA PROPORCION
train, test = train_test_split(dataset,
                               test_size = TEST_SIZE,
                               random_state = SEED,
                               stratify = dataset.AdoptionSpeed)

In [90]:
#Armo listas con features de texto y numericas
char_feats = [f for f in dataset.columns if dataset[f].dtype=='O']
numeric_feats = [f for f in dataset.columns if dataset[f].dtype!='O']

In [91]:
#Lista de features numericas
#ESTE EJEMPLO SE HACE SOLO CON VARIABLES NUMERICAS
numeric_feats

['Type',
 'Age',
 'Breed1',
 'Breed2',
 'Gender',
 'Color1',
 'Color2',
 'Color3',
 'MaturitySize',
 'FurLength',
 'Vaccinated',
 'Dewormed',
 'Sterilized',
 'Health',
 'Quantity',
 'Fee',
 'State',
 'VideoAmt',
 'PhotoAmt',
 'AdoptionSpeed',
 'AgeCategory',
 'Breed',
 'IsMultiColored',
 'name_code',
 'Is_Genuine_Name',
 'Age_log',
 'PhotoAmt_log',
 'VideoAmt_log',
 'Fee_log',
 'Gender_Age_mean',
 'Gender_Fee_mean',
 'Gender_PhotoAmt_mean',
 'Gender_VideoAmt_mean',
 'Breed_Age_mean',
 'Breed_Fee_mean',
 'Breed_PhotoAmt_mean',
 'Breed_VideoAmt_mean',
 'Gender_Breed_Age_mean',
 'Gender_Breed_Fee_mean',
 'Gender_Breed_PhotoAmt_mean',
 'Gender_Breed_VideoAmt_mean',
 'Age_to_Gender_mean_ratio',
 'Fee_to_Gender_mean_ratio',
 'Age_to_Breed_mean_ratio',
 'Fee_to_Breed_mean_ratio',
 'Age_Fee_interaction',
 'PhotoAmt_VideoAmt_interaction',
 'Gender_Fee_mean_log',
 'Breed_Fee_mean_log',
 'Gender_Breed_Fee_mean_log',
 'Age_PhotoAmt_Fee_interaction',
 'Age_VideoAmt_Fee_interaction',
 'Age_PhotoAmt'

In [92]:

#Defino features a usar en un primer modelo de prueba
features = ['Type',
 'Age',
 'Breed1',
 'Breed2',
 'Gender',
 'Color1',
 'Color2',
 'Color3',
 'MaturitySize',
 'FurLength',
 'Vaccinated',
 'Dewormed',
 'Sterilized',
 'Health',
 'Quantity',
 'Fee',
 'State',
 'VideoAmt',
 'PhotoAmt',
 #'AdoptionSpeed',
 'AgeCategory',
 'Breed',
 'IsMultiColored',
 'name_code',
 'Is_Genuine_Name',
 'Age_log',
 'PhotoAmt_log',
 'VideoAmt_log',
 'Fee_log',
 'Gender_Age_mean',
 'Gender_Fee_mean',
 'Gender_PhotoAmt_mean',
 'Gender_VideoAmt_mean',
 'Breed_Age_mean',
 'Breed_Fee_mean',
 'Breed_PhotoAmt_mean',
 'Breed_VideoAmt_mean',
 'Gender_Breed_Age_mean',
 'Gender_Breed_Fee_mean',
 'Gender_Breed_PhotoAmt_mean',
 'Gender_Breed_VideoAmt_mean',
 'Age_to_Gender_mean_ratio',
 'Fee_to_Gender_mean_ratio',
 'Age_to_Breed_mean_ratio',
 'Fee_to_Breed_mean_ratio',
 'Age_Fee_interaction',
 'PhotoAmt_VideoAmt_interaction',
 'Gender_Fee_mean_log',
 'Breed_Fee_mean_log',
 'Gender_Breed_Fee_mean_log',
 'Age_PhotoAmt_Fee_interaction',
 'Age_VideoAmt_Fee_interaction',
 'Age_PhotoAmt',
 'Age_VideoAmt',
 'Photo_Video_Ratio',
 'Fee_Age_Ratio',
 'MaturitySize_FurLength',
 'PhotoAmt_FurLength',
 'VideoAmt_FurLength',
 'Breed1_mean_Age',
 'Breed1_mean_PhotoAmt',
 'RescuerID_count',
 'description_word_count',
 'Description_Length',
 'Name_Length',
 'Name_Word_Count',
 'Name_Length_Category',
 'Health_vac',
 'Health_dew',
 'Health_ster',
 'Health_vac_dew',
 'Health_vac_ster',
 'Health_dew_ster',
 'Health_full',
 'Health_not_vac',
 'Health_not_dew',
 'Health_not_ster',
 'Health_not_vac_dew',
 'Health_not_vac_ster',
 'Health_not_dew_ster',
 'Health_none',
# 'prediction_class_0',
# 'prediction_class_1',
# 'prediction_class_2',
# 'prediction_class_3',
# 'prediction_class_4',
 'RescuerID_mean_Age',
 'RescuerID_std_Age',
 'RescuerID_mean_Fee',
 'RescuerID_std_Fee',
 'RescuerID_mean_PhotoAmt',
 'RescuerID_mean_VideoAmt',
 'RescuerID_name_ratio',
 'RescuerID_most_common_breed',
 'RescuerID_young_ratio',
 'RescuerID_full_health_ratio',
 'RescuerID_mean_description_length',
 'RescuerID_dog_ratio',
 'RescuerID_breed_specialization']
 #'prediction_class_0',
 #'prediction_class_1',
 #'prediction_class_2',
 #'prediction_class_3',
 #'prediction_class_4']

label = 'AdoptionSpeed'

In [93]:
train

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,...,RescuerID_std_Fee,RescuerID_mean_PhotoAmt,RescuerID_mean_VideoAmt,RescuerID_name_ratio,RescuerID_most_common_breed,RescuerID_young_ratio,RescuerID_full_health_ratio,RescuerID_mean_description_length,RescuerID_dog_ratio,RescuerID_breed_specialization
10070,1,The Adorable Trio,2,307,307,1,1,0,0,2,...,0.000000,6.000000,0.000000,1.000000,307,1.000000,0.000000,355.500000,1.000000,1.000000
13397,1,Perky,12,307,0,2,2,0,0,1,...,0.000000,2.119048,0.000000,0.928571,307,0.595238,0.357143,140.166667,0.547619,0.404762
14656,1,Bernard Boy,2,307,307,1,1,2,7,2,...,56.558987,5.871795,0.025641,1.000000,307,0.948718,0.102564,407.564103,0.948718,0.897436
5596,1,Oreo 1,2,307,307,1,3,0,0,2,...,63.672346,3.140351,0.000000,1.000000,307,0.916667,0.057018,14.271930,1.000000,0.916667
13463,2,Anak Kucing,4,265,0,3,1,2,0,2,...,NaN,2.000000,0.000000,1.000000,265,1.000000,0.000000,20.000000,0.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9981,2,Maxine,8,265,299,2,1,2,0,2,...,NaN,2.000000,0.000000,1.000000,265,1.000000,0.000000,340.000000,0.000000,1.000000
10249,1,Browny,2,307,0,2,6,7,0,1,...,NaN,1.000000,0.000000,1.000000,307,1.000000,0.000000,313.000000,1.000000,1.000000
12468,1,Midnight,2,307,307,1,1,0,0,2,...,0.000000,2.000000,0.000000,1.000000,307,1.000000,0.000000,283.000000,1.000000,1.000000
1081,1,Boxer,2,307,307,1,1,2,7,2,...,0.000000,2.305970,0.007463,1.000000,307,0.895522,0.044776,242.029851,0.970149,0.895522


In [94]:
#Genero dataframes de train y test con sus respectivos targets
X_train = train[features]
y_train = train[label]

X_test = test[features]
y_test = test[label]

In [95]:
#Entreno un modelo inicial sin modificar hiperparametros. Solamente especifico el numero de clases y el tipo de modelo como clasificacoión
#HACEMOS LGBM POR DEFECTO COMO VIENE
lgb_params = params = {
                        'objective': 'multiclass',#EL PROBLEMA ES UNA MULTICPLASE 
                        'num_class': len(y_train.unique())
                        }


#genero el objeto Dataset que debo pasarle a lightgbm para que entrene
lgb_train_dataset = lgb.Dataset(data=X_train,
                                label=y_train)

#entreno el modelo con los parametros por defecto
lgb_model = lgb.train(lgb_params,
                      lgb_train_dataset)

In [96]:
#Obtengo las predicciones sobre el set de test. El modelo me da una lista de probabilidades para cada clase y tomo la clase con mayor probabilidad con la funcion argmax
y_pred = lgb_model.predict(X_test).argmax(axis=1)#CON ARGMAX OBTENGO LA PREDICCION MAS GRANDE DE LAS CATEGORIAS OSEA LA GANADORA

#Calculo el Kappa
cohen_kappa_score(y_test,y_pred, weights = 'quadratic')
#MI MODELO DA UN 0.31 DE SCORE 
#RECORDAR QUE AL SER UN MULTICLASE EL PREDICT ME TRAE 5 COLUMNAS ( PROBABILIDAD DEL MODELO A CADA UNA DE LAS CLASES )




np.float64(0.3858432897143127)

In [97]:
#Muestro la matriz de confusión
display(plot_confusion_matrix(y_test,y_pred))
#SI OBTENGO UNA DIAGONAL MUY FUERTE AMARILLA ES PQ PREDIJO BIEN EL MODELO 

In [98]:
#Vamos a ponewr en perspectiva el score de Kappa


#Cual es el score perfecto? Evaluo la clase real contra si misma. Es decir, el caso en que el modelo establece todas las clases en su valor real
cohen_kappa_score(y_test,y_test, weights = 'quadratic')

np.float64(1.0)

In [99]:
#Como se veria la matriz de confusión PERO ESTA ES LA PERFECTA
display(plot_confusion_matrix(y_test,y_test))

In [100]:
#SIMULACIONES

In [101]:
#Pruebo un modelo alternativo donde en vez de usar la version multiclass real de lightGBM utilizo One vs All

lgb_params = params = {
                        'objective': 'multiclassova', # ESTA HACE MUCHAS LOGITICAS COMBINADAS PERO SIEMPRE DECIDE QUEDARSE CON UNA 
                        'num_class': len(y_train.unique())
                        }


lgb_train_dataset = lgb.Dataset(data=X_train,
                                label=y_train)


lgb_model = lgb.train(lgb_params,
                      lgb_train_dataset)

In [102]:
#MAtriz de confusion y Kappa dfe OVA
y_pred = lgb_model.predict(X_test).argmax(axis=1)

display(plot_confusion_matrix(y_test,y_pred))

{'kappa':cohen_kappa_score(y_test,
                y_pred,
                weights = 'quadratic'),
 'accuracy':accuracy_score(y_test,y_pred),
 'balanced_accuracy':balanced_accuracy_score(y_test,y_pred)}




{'kappa': np.float64(0.3736583672862005),
 'accuracy': 0.4381460486828943,
 'balanced_accuracy': np.float64(0.3647091257530836)}

In [103]:
#USAR MULTINOVA MEJORA KAPPA OSEA MI MODELO SE ACERCA MAS AL ORIG

## Optimizacion de hiperparametros modelo train/test

In [104]:
#AHORA SUMAMOS HIPERPARAMETROS PARA VER QUE PASA CON LGBM

In [105]:

#Funcion que vamos a optimizar. Optuna requiere que usemos el objeto trial para generar los parametros a optimizar
def lgb_objective(trial):
    #PArametros para LightGBM
    lgb_params = {      
                        #PArametros fijos
                        'objective': 'multiclass',
                        'verbosity':-1,
                        'num_class': len(y_train.unique()),
                        #Hiperparametros a optimizar utilizando suggest_float o suggest_int segun el tipo de dato
                        #Se indica el nombre del parametro, valor minimo, valor maximo 
                        #en elgunos casos el parametro log=True para parametros que requieren buscar en esa escala
                        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
                        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
                        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
                        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
                        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
                        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
                        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
                        } 

    #Genero objeto dataset de entrenamiento
    lgb_train_dataset = lgb.Dataset(data=X_train,
                                    label=y_train)

    #ajuste de modelo
    lgb_model = lgb.train(lgb_params,
                        lgb_train_dataset)
    
    #Devuelvo el score en test
    return(cohen_kappa_score(y_test,lgb_model.predict(X_test).argmax(axis=1),
                             weights = 'quadratic'))

In [106]:
#Defino el estudio a optimizar
study = optuna.create_study(direction='maximize', #buscamos maximizar la metrica
                            storage="sqlite:///work/db.sqlite3",  # Specify the storage URL here.
                            study_name="04 - LGB Multiclass", #nombre del experimento
                            load_if_exists=True) #continuar si ya existe

#Corremos 100 trials para buscar mejores parametros
study.optimize(lgb_objective, n_trials=100)

[I 2024-09-30 18:14:09,030] Using an existing study with name '04 - LGB Multiclass' instead of creating a new one.
[I 2024-09-30 18:14:12,669] Trial 2197 finished with value: 0.37202624855581423 and parameters: {'lambda_l1': 5.052444787110685e-05, 'lambda_l2': 8.828256694410191e-08, 'num_leaves': 10, 'feature_fraction': 0.9894632002829816, 'bagging_fraction': 0.549160285269878, 'bagging_freq': 6, 'min_child_samples': 17}. Best is trial 200 with value: 1.0.
[I 2024-09-30 18:14:14,122] Trial 2198 finished with value: 0.368926723528364 and parameters: {'lambda_l1': 0.00022506811570793548, 'lambda_l2': 1.6758964124678567e-07, 'num_leaves': 15, 'feature_fraction': 0.999821013437169, 'bagging_fraction': 0.5819913387145501, 'bagging_freq': 5, 'min_child_samples': 13}. Best is trial 200 with value: 1.0.
[I 2024-09-30 18:14:16,525] Trial 2199 finished with value: 0.379802714214006 and parameters: {'lambda_l1': 0.00010787751839338808, 'lambda_l2': 5.563276545963549e-08, 'num_leaves': 39, 'featur

In [107]:
#corriendo esto en terminal deberia ver el dashboard 
#optuna-dashboard sqlite:///work/db.sqlite3 --artifact-dir /work/optuna_artifacts --port 8081

In [108]:
#Obtenemos mejor resultado
study.best_params

{'lambda_l1': 0.00016911420117746327,
 'lambda_l2': 3.0291532676782044e-07,
 'num_leaves': 23,
 'feature_fraction': 0.9710839320963843,
 'bagging_fraction': 0.6006057268679922,
 'bagging_freq': 6,
 'min_child_samples': 16}

In [109]:
#Vamos a replicar el resultado de la optimizacion reentrenando el modelo con el mejor conjunto de hiperparametros
#Generamos parametros incluyendo los fijos y la mejor solución que encontro optuna
lgb_params = {
    'objective': 'multiclass',
    'verbosity': -1,
    'num_class': len(y_train.unique())
}

# Update lgb_params with study.best_params
lgb_params.update(study.best_params)

lgb_train_dataset = lgb.Dataset(data=X_train,
                                label=y_train)


#Entreno
lgb_model = lgb.train(lgb_params,
                    lgb_train_dataset)

#Muestro matriz de confusion y kappa
display(plot_confusion_matrix(y_test,lgb_model.predict(X_test).argmax(axis=1)))

cohen_kappa_score(y_test,lgb_model.predict(X_test).argmax(axis=1),
                             weights = 'quadratic')


np.float64(0.3837502657680879)

In [110]:
#A PARTIR DE OPTUNA (OPTIMIZACION BAYESIANA) ES LO MEJOR TENGO ENTENDIDO, ECONTRE LOS MEJORES VALORES HIPERPARMTROS
#Y ESO MEJORO KAPPA 

In [111]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Configurar visualizaciones
sns.set(style="whitegrid")

In [112]:
import matplotlib.pyplot as plt
import pandas as pd
import textwrap

# Obtener la importancia de las características
importance = lgb_model.feature_importance(importance_type='gain')
feature_names = X_train.columns

# Crear un DataFrame para mostrar las importancias
feature_importance_df = pd.DataFrame({'feature': feature_names, 'importance': importance})

# Ordenar el DataFrame por importancia
feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)

# Función para envolver el texto
def wrap_labels(label, max_width=20):
    return '\n'.join(textwrap.wrap(label, max_width))

# Aplicar la función de envoltura a las etiquetas de características
feature_importance_df['wrapped_feature'] = feature_importance_df['feature'].apply(wrap_labels)

# Crear la figura y los ejes
fig, ax = plt.subplots(figsize=(12, len(feature_importance_df) * 0.4))

# Crear el gráfico de barras horizontales
bars = ax.barh(feature_importance_df['wrapped_feature'], feature_importance_df['importance'])

# Configurar el título y las etiquetas
ax.set_title('Importancia de las Características')
ax.set_xlabel('Importancia')
ax.set_ylabel('Características')

# Ajustar el espacio entre las etiquetas del eje y
plt.subplots_adjust(left=0.3)

# Invertir el orden del eje y para que la característica más importante esté arriba
ax.invert_yaxis()

# Ajustar los márgenes
plt.tight_layout()

# Mostrar el gráfico
plt.show()

## predecir en archivo test.csv

In [45]:
df_test = pd.read_csv(PATH_TO_TEST)
#df_test

#FEATURE ENGINEERING
#Nuevas variables 

dataset=feature_engineering(df_test)

#predict
df_predict_test=lgb_model.predict(dataset[features]).argmax(axis=1)



In [46]:
pd.Series(df_predict_test).value_counts(normalize=True)

4    0.400554
2    0.247482
1    0.238671
3    0.109013
0    0.004280
dtype: float64

In [47]:
#add predict colum to the dataset
dataset['AdoptionSpeed'] = df_predict_test

In [48]:
#save for kaggle competition   
dataset[['PetID', 'AdoptionSpeed']].to_csv('submission.csv', index=False)


## Modelo con cross validation y conjunto de test

In [41]:
#Genero una metrica para que lightGBM haga la evaluación y pueda hacer early_stopping en el cross validation
def lgb_custom_metric_kappa(dy_pred, dy_true):
    metric_name = 'kappa'
    value = cohen_kappa_score(dy_true.get_label(),dy_pred.argmax(axis=1),weights = 'quadratic')
    is_higher_better = True
    return(metric_name, value, is_higher_better)

#Funcion objetivo a optimizar. En este caso vamos a hacer 5fold cv sobre el conjunto de train. 
# El score de CV es el objetivo a optimizar. Ademas vamos a usar los 5 modelos del CV para estimar el conjunto de test,
# registraremos en optuna las predicciones, matriz de confusion y el score en test.
# CV Score -> Se usa para determinar el rendimiento de los hiperparametros con precision 
# Test Score -> Nos permite testear que esta todo OK, no use (ni debo usar) esos datos para nada en el entrenamiento 
# o la optimizacion de hiperparametros

def cv_es_lgb_objective(trial):

    #PArametros para LightGBM
    lgb_params = {      
                        #PArametros fijos
                        'objective': 'multiclass',
                        'verbosity':-1,
                        'num_class': len(y_train.unique()),
                        #Hiperparametros a optimizar utilizando suggest_float o suggest_int segun el tipo de dato
                        #Se indica el nombre del parametro, valor minimo, valor maximo 
                        #en elgunos casos el parametro log=True para parametros que requieren buscar en esa escala
                        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
                        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
                        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
                        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
                        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
                        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
                        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
                        } 

    #Voy a generar estimaciones de los 5 modelos del CV sobre los datos test y los acumulo en la matriz scores_ensemble
    scores_ensemble = np.zeros((len(y_test),len(y_train.unique())))

    #Score del 5 fold CV inicializado en 0
    score_folds = 0

    #Numero de splits del CV
    n_splits = 5

    #Objeto para hacer el split estratificado de CV
    skf = StratifiedKFold(n_splits=n_splits)

    for i, (if_index, oof_index) in enumerate(skf.split(X_train, y_train)):
        
        #Dataset in fold (donde entreno) 
        lgb_if_dataset = lgb.Dataset(data=X_train.iloc[if_index],
                                        label=y_train.iloc[if_index],
                                        free_raw_data=False)
        
        #Dataset Out of fold (donde mido la performance del CV)
        lgb_oof_dataset = lgb.Dataset(data=X_train.iloc[oof_index],
                                        label=y_train.iloc[oof_index],
                                        free_raw_data=False)

        #Entreno el modelo
        lgb_model = lgb.train(lgb_params,
                                lgb_if_dataset,
                                valid_sets=lgb_oof_dataset,
                                callbacks=[lgb.early_stopping(10, verbose=False)],
                                feval = lgb_custom_metric_kappa
                                )
        
        #Acumulo los scores (probabilidades) de cada clase para cada uno de los modelos que determino en los folds
        #Se predice el 20% de los datos que separe para tes y no uso para entrenar en ningun fold
        scores_ensemble = scores_ensemble + lgb_model.predict(X_test)
        
        #Score del fold (registros de dataset train que en este fold quedan out of fold)
        score_folds = score_folds + cohen_kappa_score(y_train.iloc[oof_index], 
                                                            lgb_model.predict(X_train.iloc[oof_index]).argmax(axis=1),weights = 'quadratic')/n_splits


    #Guardo prediccion del trial sobre el conjunto de test
    # Genero nombre de archivo
    predicted_filename = os.path.join(PATH_TO_TEMP_FILES,f'test_{trial.study.study_name}_{trial.number}.joblib')
    # Copia del dataset para guardar la prediccion
    predicted_df = test.copy()
    # Genero columna pred con predicciones sumadas de los 5 folds
    predicted_df['pred'] = [scores_ensemble[p,:] for p in range(scores_ensemble.shape[0])]
    # Grabo dataframe en temp_artifacts
    dump(predicted_df, predicted_filename)
    # Indico a optuna que asocie el archivo generado al trial
    upload_artifact(trial, predicted_filename, artifact_store)    

    #Grabo natriz de confusion
    #Nombre de archivo
    cm_filename = os.path.join(PATH_TO_TEMP_FILES,f'cm_{trial.study.study_name}_{trial.number}.jpg')
    #Grabo archivo
    plot_confusion_matrix(y_test,scores_ensemble.argmax(axis=1)).write_image(cm_filename)
    #Asocio al trial
    upload_artifact(trial, cm_filename, artifact_store)

    #Determino score en conjunto de test y asocio como metrica adicional en optuna
    test_score = cohen_kappa_score(y_test,scores_ensemble.argmax(axis=1),weights = 'quadratic')
    trial.set_user_attr("test_score", test_score)

    #Devuelvo score del 5fold cv a optuna para que optimice en base a eso
    return(score_folds)

In [42]:
#Inicio el store de artefactos (archivos) de optuna
artifact_store = FileSystemArtifactStore(base_path=PATH_TO_OPTUNA_ARTIFACTS)

#Genero estudio
study = optuna.create_study(direction='maximize',
                            storage="sqlite:///work/db.sqlite3",  # Specify the storage URL here.
                            study_name="04 - LGB Multiclass CV",
                            load_if_exists = True)
#Corro la optimizacion
study.optimize(cv_es_lgb_objective, n_trials=100)

<ipython-input-42-3ba8469af45f>:2: ExperimentalWarning:

FileSystemArtifactStore is experimental (supported from v3.3.0). The interface can change in the future.

[I 2024-09-14 12:14:58,409] Using an existing study with name '04 - LGB Multiclass CV' instead of creating a new one.
<ipython-input-41-1c6539846d38>:86: ExperimentalWarning:

upload_artifact is experimental (supported from v3.3.0). The interface can change in the future.

<ipython-input-41-1c6539846d38>:94: ExperimentalWarning:

upload_artifact is experimental (supported from v3.3.0). The interface can change in the future.

[I 2024-09-14 12:15:22,817] Trial 671 finished with value: 0.37184665533691497 and parameters: {'lambda_l1': 0.5388084648637167, 'lambda_l2': 1.2860602375541024, 'num_leaves': 6, 'feature_fraction': 0.41325944952074783, 'bagging_fraction': 0.43724914537542203, 'bagging_freq': 1, 'min_child_samples': 26}. Best is trial 45 with value: 1.0.
<ipython-input-41-1c6539846d38>:86: ExperimentalWarning:

upload_ar

KeyboardInterrupt: 

Para ver el optuna dashboard tengo que correr este comando en la terminal

In [39]:
!pip install kaleido


In [40]:
!optuna-dashboard sqlite://work/db.sqlite3 --artifact-dir ../work/optuna_artifacts --port 8081

Traceback (most recent call last):
  File "c:\users\s1093678\anaconda3\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\users\s1093678\anaconda3\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\s1093678\Anaconda3\Scripts\optuna-dashboard.exe\__main__.py", line 7, in <module>
  File "c:\users\s1093678\anaconda3\lib\site-packages\optuna_dashboard\_cli.py", line 119, in main
    storage = get_storage(args.storage, storage_class=args.storage_class)
  File "c:\users\s1093678\anaconda3\lib\site-packages\optuna_dashboard\_storage_url.py", line 59, in get_storage
    return guess_storage_from_url(storage)
  File "c:\users\s1093678\anaconda3\lib\site-packages\optuna_dashboard\_storage_url.py", line 84, in guess_storage_from_url
    return get_rdb_storage(storage_url)
  File "c:\users\s1093678\anaconda3\lib\site-packages\optuna_dashboard\_storage_url.py", line 91, in get_rdb_storage
    return RDBStorage(st

In [41]:
#para proxima clase hacer feature engeneering (agregar feature, health , colores, etc) pq hasta aca solo s ehizo con los variables numericas
#TENER UN MODELO CANDIDATO TABULAR PARA SEGUIR ADELANTE 
#VAMOS A VER LA EJECUCION DE UN MODELO PARA BUCAR HIPERPARAMETROS ETC 
#DAR CON UN MODELO GANADOR
